## ANALISIS DE LOS DATOS DE CACAO

### Importar datos y establecer variables de interes

In [55]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

In [56]:
df = pd.read_csv('/content/drive/MyDrive/romina 2024/pruebas cacao/solocacao1.csv')

In [57]:
variables = ['CAC_INTERNACIONAL_TN', 'CAC_EXPORTACIONES_TN' ,
                   'CAC_EXPORTACIONES_FOB', 'CAC_PRECIO_DE_EXPORTACION_ECUADOR']

#### Calculo Multicolinealidad VIF

In [58]:


# Definir las variables de interés
variables = ['CAC_INTERNACIONAL_TN', 'CAC_EXPORTACIONES_TN', 'CAC_EXPORTACIONES_FOB', 'CAC_PRECIO_DE_EXPORTACION_ECUADOR']

# Filtrar el DataFrame para mantener solo las variables de interés
df_selected = df[variables]

# Calcular VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = df_selected.columns
vif_data['VIF'] = [variance_inflation_factor(df_selected.values, i) for i in range(len(df_selected.columns))]

# Mostrar resultados
print("VIF:")
vif_data




VIF:


,Variable,VIF
0,CAC_INTERNACIONAL_TN,69.803490
1,CAC_EXPORTACIONES_TN,9.179040
2,CAC_EXPORTACIONES_FOB,18.418051
3,CAC_PRECIO_DE_EXPORTACION_ECUADOR,70.066960


Los datos presentan mucha multicolinealidad, por lo tanto se va a proceder a hacer un análisis de componentes principales para tratarla

In [59]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[variables]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicación de PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Convertir las componentes principales a un DataFrame
df_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

# Calcular el VIF
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

# Calcular VIF para las componentes principales
vif_df = calculate_vif(df_pca)

# Mostrar resultados

vif_df

,feature,VIF
0,PC1,1.0
1,PC2,1.0
2,PC3,1.0
3,PC4,1.0


Ahora que se hizo el PCA se trató la multicolinealidad disminuyendola a 1, ahora se puede continuar

#### Shapiro Wilk

Ahora se hace un analisis de normalidad de datos

In [60]:
from scipy.stats import boxcox
from scipy.stats import shapiro
# Calcular el test de Shapiro-Wilk para las variables sin transformaciones previas
shapiro_results = {col: shapiro(df[col]) for col in variables}

# Convertir los resultados a un DataFrame
shapiro_results_df = pd.DataFrame(shapiro_results, index=['Statistic', 'p-value'])

print(shapiro_results_df)


           CAC_INTERNACIONAL_TN  CAC_EXPORTACIONES_TN  CAC_EXPORTACIONES_FOB  \
Statistic          5.273161e-01          9.084107e-01           7.701155e-01   
p-value            4.294682e-18          4.521224e-07           1.529353e-12   

           CAC_PRECIO_DE_EXPORTACION_ECUADOR  
Statistic                       5.362921e-01  
p-value                         6.238258e-18  


Datos no presentan normalidad, para tratar estos datos y disminuir los problemas de normalidad se hace un analisis BoxCox

In [61]:
from scipy.stats import boxcox
from scipy.stats import shapiro
# Crear una copia del dataframe original
df_adjusted_mean = df.copy()

# Calcular el promedio de la cuarta variable
mean_value = df_adjusted_mean['CAC_PRECIO_DE_EXPORTACION_ECUADOR'].mean()

# Reemplazar los últimos 5 registros de la cuarta variable con el valor promedio
df_adjusted_mean.loc[df_adjusted_mean.index[-5:], 'CAC_PRECIO_DE_EXPORTACION_ECUADOR'] = mean_value

# Aplicar la transformación de Box-Cox a las variables originales en el dataframe ajustado
boxcox_transformed_mean = {}
for var in variables:
    min_value = df_adjusted_mean[var].min()
    if min_value <= 0:
        adjusted_data = df_adjusted_mean[var] - min_value + 1
    else:
        adjusted_data = df_adjusted_mean[var]
    transformed_data, _ = boxcox(adjusted_data)
    boxcox_transformed_mean[var] = transformed_data

# Convertir los datos transformados a un DataFrame
df_boxcox_mean = pd.DataFrame(boxcox_transformed_mean)

# Calcular el test de Shapiro-Wilk para cada variable transformada en el dataframe ajustado
shapiro_results_boxcox_mean = {col: shapiro(df_boxcox_mean[col]) for col in df_boxcox_mean.columns}

# Convertir los resultados a un DataFrame
shapiro_results_boxcox_mean_df = pd.DataFrame(shapiro_results_boxcox_mean, index=['Statistic', 'p-value'])

shapiro_results_boxcox_mean_df


,CAC_INTERNACIONAL_TN,CAC_EXPORTACIONES_TN,CAC_EXPORTACIONES_FOB,CAC_PRECIO_DE_EXPORTACION_ECUADOR
Statistic,0.981395,0.992332,0.992033,0.985081
p-value,0.090560,0.742285,0.713844,0.199840


Después de aplicar la técnica boxcox se se arreglaron los problemas de normalidad

#### Calculo de Datos Atípicos

Primero se procede a calcular el nivel crítico para determinar el umbral de referencia de los datos atípicos teniendo en cuenta el numero de variables de nuestro interés

In [62]:
from scipy.stats import chi2

# Número de variables (grados de libertad)
df_degrees_of_freedom = len(df_pca.columns)

# Nivel de confianza (1 - alpha)
alpha = 0.05

# Calcular el valor crítico de la distancia de Mahalanobis
critical_value = chi2.ppf((1 - alpha), df_degrees_of_freedom)
print('El valor critico para determinar prescencia de datos atipicos es:')
critical_value


El valor critico para determinar prescencia de datos atipicos es:


9.487729036781154

Ahora se calcula y se hace un resumen de las distancias de Mahalanobis, si algún valor supera el nivel critico, se puede considerar datos atípicos

In [63]:
# Paso 1: Importar bibliotecas necesarias
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2

# Calcular la media y la matriz de covarianza
mean = df_pca.mean().values
cov_matrix = np.cov(df_pca, rowvar=False)

# Invertir la matriz de covarianza
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Calcular la distancia de Mahalanobis para cada punto
distances = df_pca.apply(lambda row: mahalanobis(row, mean, inv_cov_matrix), axis=1)

distances_summary = {
    "Distancia Promedio": [distances.mean()],
    "Desviación Estándar": [distances.std()],
    "Valor Mínimo": [distances.min()],
    "Valor Máximo": [distances.max()],
}

# Crear un DataFrame con los resultados
distances_summary_df = pd.DataFrame(distances_summary)

# Mostrar el resumen
print(distances_summary_df)


   Distancia Promedio  Desviación Estándar  Valor Mínimo  Valor Máximo
0            1.432758             1.389331      0.282676       9.38011


La distancia promedio es 1.43, no hay datos atípicos en general en el conjunto de los datos, el valor máximo registrado no supera el umbral critico, por lo que no existen problemas de datos atípicos

#### CALCULO Y EVALUACION DEL MODELO VEC

In [77]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from sklearn.preprocessing import StandardScaler
from scipy import stats
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen




##### Pruebas ADF Y KPSS

In [78]:
variables = ['CAC_INTERNACIONAL_TN','CAC_PRECIO_DE_EXPORTACION_ECUADOR']

# Añadir los componentes principales al DataFrame original
# Prueba de ADF
def adf_test(series, title=''):
    result = adfuller(series, autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(f'Augmented Dickey-Fuller Test on "{title}"\n')
    print(out.to_string())
    print('\n')

# Prueba de KPSS
def kpss_test(series, title=''):
    result = kpss(series, regression='c')
    labels = ['KPSS Test Statistic', 'p-value', '# Lags Used']
    out = pd.Series(result[0:3], index=labels)
    for key, value in result[3].items():
        out[f'Critical Value ({key})'] = value
    print(f'KPSS Test on "{title}"\n')
    print(out.to_string())
    print('\n')

# Pruebas de ADF y KPSS para ambas variables
for variable in variables:
    adf_test(df[variable], title=variable)
    kpss_test(df[variable], title=variable)




Augmented Dickey-Fuller Test on "CAC_INTERNACIONAL_TN"

ADF Test Statistic        1.821601
p-value                   0.998391
# Lags Used               6.000000
# Observations Used     115.000000
Critical Value (1%)      -3.488535
Critical Value (5%)      -2.887020
Critical Value (10%)     -2.580360


KPSS Test on "CAC_INTERNACIONAL_TN"

KPSS Test Statistic      0.410004
p-value                  0.072843
# Lags Used              5.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000


Augmented Dickey-Fuller Test on "CAC_PRECIO_DE_EXPORTACION_ECUADOR"

ADF Test Statistic        6.817075
p-value                   1.000000
# Lags Used               0.000000
# Observations Used     121.000000
Critical Value (1%)      -3.485585
Critical Value (5%)      -2.885739
Critical Value (10%)     -2.579676


KPSS Test on "CAC_PRECIO_DE_EXPORTACION_ECUADOR"

KPSS Test Statistic      0.405470
p-value             


No hay problemas de estacionalidad porque los resultados de las pruebas indican que ambas series no son estacionarias. El ADF test muestra p-valores altos (cercanos a 1), lo que significa que no se puede rechazar la hipótesis nula de que las series tienen una raíz unitaria (no son estacionarias). Por otro lado, el test KPSS, aunque tiene p-valores cercanos a 0.05, no llega a rechazar la hipótesis nula de que las series son estacionarias. Esto sugiere que, aunque no son estacionarias.

##### Prueba Johansen

In [66]:
# Prueba de Cointegración de Johansen
# Eliminar la columna Fecha
pca_df_numeric = pca_df.drop(columns=['Fecha'])

# Realizar la prueba de cointegración
coint_test = coint_johansen(pca_df_numeric, det_order=0, k_ar_diff=1)
print(f'Cointegration Test Summary:\nEigenvalues: {coint_test.eig}\n')

Cointegration Test Summary:
Eigenvalues: [0.40147776 0.0772182 ]



hay evidencia de cointegración, sugiriendo una relación a largo plazo entre las variables de precios.

In [67]:
# Prueba de Cointegración de Johansen


# aqui elimino la columna Fecha
pca_df_numeric = pca_df.drop(columns=['Fecha'])

# Realizar la prueba de cointegración
coint_test = coint_johansen(pca_df_numeric, det_order=0, k_ar_diff=1)

# Resultados detallados
def summary_johansen(coint_test):
    summary = {}
    summary['r'] = range(len(coint_test.lr1))
    summary['Trace Statistic'] = coint_test.lr1
    summary['Critical Value (90%)'] = coint_test.cvt[:, 0]
    summary['Critical Value (95%)'] = coint_test.cvt[:, 1]
    summary['Critical Value (99%)'] = coint_test.cvt[:, 2]
    summary['Max Eigen Statistic'] = coint_test.lr2
    summary['Critical Value Max (90%)'] = coint_test.cvm[:, 0]
    summary['Critical Value Max (95%)'] = coint_test.cvm[:, 1]
    summary['Critical Value Max (99%)'] = coint_test.cvm[:, 2]
    return pd.DataFrame(summary)

johansen_summary = summary_johansen(coint_test)
print(johansen_summary)

   r  Trace Statistic  Critical Value (90%)  Critical Value (95%)  \
0  0        71.238489               13.4294               15.4943   
1  1         9.643497                2.7055                3.8415   

   Critical Value (99%)  Max Eigen Statistic  Critical Value Max (90%)  \
0               19.9349            61.594991                   12.2971   
1                6.6349             9.643497                    2.7055   

   Critical Value Max (95%)  Critical Value Max (99%)  
0                   14.2639                   18.5200  
1                    3.8415                    6.6349  


 Ambas pruebas (Trace y Max Eigen) indican que existe cointegración entre las variables de precios, hay relación a largo plazo en las variables de precios.

#### Tabla Resumen VEC

In [68]:
from statsmodels.tsa.vector_ar.vecm import VECM

# Ajustar el modelo VECM
vecm = VECM(pca_df_numeric, k_ar_diff=1, coint_rank=1)
vecm_fit = vecm.fit()

# Resumen del modelo
print(vecm_fit.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation PC1
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
L1.PC1         0.5448      0.154      3.535      0.000       0.243       0.847
L1.PC2         0.0161      0.155      0.104      0.917      -0.288       0.320
Det. terms outside the coint. relation & lagged endog. parameters for equation PC2
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
L1.PC1         0.0773      0.132      0.587      0.557      -0.181       0.335
L1.PC2         0.1644      0.133      1.241      0.215      -0.095       0.424
                Loading coefficients (alpha) for equation PC1                 
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------

##### Prueba Ljung Box

In [69]:
import pandas as pd
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.vector_ar.vecm import VECM

# Ajustar el modelo VECM
vecm = VECM(pca_df_numeric, k_ar_diff=1, coint_rank=1)
vecm_fit = vecm.fit()

# Obtener las residuales del modelo y convertirlas en un DataFrame
residuales = pd.DataFrame(vecm_fit.resid, columns=pca_df_numeric.columns)

# Calcular la prueba de Ljung-Box para cada serie residual
resultados_ljung_box = {}
for col in residuales.columns:
    ljung_box_test = acorr_ljungbox(residuales[col], lags=[10], return_df=True)
    resultados_ljung_box[col] = ljung_box_test

# Mostrar los resultados
for key, value in resultados_ljung_box.items():
    print(f"Resultados de la prueba de Ljung-Box para {key}:")
    print(value)
    print("\n")


Resultados de la prueba de Ljung-Box para PC1:
     lb_stat  lb_pvalue
10  9.962627   0.443778


Resultados de la prueba de Ljung-Box para PC2:
     lb_stat  lb_pvalue
10  5.421979   0.861268




Ambos resultados indican que no hay autocorrelacion en los residuos del modelo indicando que VEC se ajusta bien a los datos

##### Durbin Watson

In [70]:
from statsmodels.stats.stattools import durbin_watson

# Calcular los residuos del modelo VECM
residuals = vecm_fit.resid

# Calcular el estadístico de Durbin-Watson para cada serie de residuos
dw_stats = durbin_watson(residuals)

# Crear un DataFrame para mostrar los resultados
dw_results = pd.DataFrame({'Variable': ['PC1', 'PC2'], 'Durbin-Watson': dw_stats})
print(dw_results)

  Variable  Durbin-Watson
0      PC1       2.050058
1      PC2       1.981509


En las variables de precios ambos durbin watson tiene valores cercanos a 2, lo que indica que no existen problemas de autocorrelacion en los residuos, indicando una gran performance del modelo

##### Predicciones VEC

In [71]:
forecast_steps = 12
vecm_forecast = vecm_fit.predict(steps=forecast_steps)

# Crear un DataFrame para mostrar los resultados del pronóstico
forecast_index = pd.date_range(start=df['Fecha'].iloc[-1], periods=forecast_steps + 1, freq='M')[1:]
forecast_df = pd.DataFrame(vecm_forecast, index=forecast_index, columns=['PC1', 'PC2'])

print(forecast_df)

                 PC1       PC2
2024-06-30  8.977879 -0.399434
2024-07-31  9.204240 -0.395947
2024-08-31  9.361836 -0.483882
2024-09-30  9.455447 -0.514565
2024-10-31  9.507047 -0.515774
2024-11-30  9.536746 -0.516966
2024-12-31  9.554652 -0.520273
2025-01-31  9.565363 -0.522563
2025-02-28  9.571634 -0.523575
2025-03-31  9.575296 -0.524066
2025-04-30  9.577452 -0.524389
2025-05-31  9.578726 -0.524600


Muy importante recordar que estos valores todavía estan en formato PCA, técnica que se usó para tratar la multicolinealidad, ahora con estos resultados finales se procede a convertirlos a la medida original (dólares)

In [72]:

variables_final = df[['CAC_INTERNACIONAL_TN', 'CAC_PRECIO_DE_EXPORTACION_ECUADOR']]

# Estandarizar el conjunto final de variables
scaler = StandardScaler()
variables_final_scaled = scaler.fit_transform(variables_final)

#resultado L1
l_un = 0.5
resid = 0.18

# Aplicar PCA al conjunto final de variables
pca_final = PCA(n_components=variables_final.shape[1])
variables_final_pca = pca_final.fit_transform(variables_final_scaled)

# Convertir los componentes PCA a un DataFrame
variables_final_pca_df = pd.DataFrame(variables_final_pca, columns=[f'PC{i+1}' for i in range(variables_final_pca.shape[1])])

# Definir el modelo VECM utilizando los componentes PCA
vecm_final_pca = VECM(variables_final_pca_df, k_ar_diff=1, coint_rank=1, deterministic='co')

# Ajustar el modelo
vecm_final_pca_fit = vecm_final_pca.fit()

# Calcular los estadísticos de Durbin-Watson para cada ecuación en el VECM
dw_stats_final_pca = [durbin_watson(vecm_final_pca_fit.resid[:, i]) for i in range(vecm_final_pca_fit.neqs)]

# Pronóstico con VECM para 12 pasos adelante
forecast_final_pca_steps = 12
forecast_final_pca = vecm_final_pca_fit.predict(steps=forecast_final_pca_steps)

# Convertir el pronóstico a un DataFrame para mejor legibilidad
forecast_final_pca_df = pd.DataFrame(forecast_final_pca, columns=variables_final_pca_df.columns)

# Invertir la transformación PCA para volver a la escala original
forecast_final_reconstructed = pca_final.inverse_transform(forecast_final_pca)

# Invertir la estandarización para volver a la escala original
forecast_final_original_scale = scaler.inverse_transform(forecast_final_reconstructed)

# Convertir a DataFrame para mejor legibilidad
forecast_final_original_scale_df = pd.DataFrame(forecast_final_original_scale, columns=variables_final.columns)

# Mostrar los resultados
print("Durbin-Watson statistics:", dw_stats_final_pca)
print("Forecast (Original Scale):")
print(forecast_final_original_scale_df)

Durbin-Watson statistics: [2.0152651822735397, 1.9878186767203974]
Forecast (Original Scale):
    CAC_INTERNACIONAL_TN  CAC_PRECIO_DE_EXPORTACION_ECUADOR
0            9828.573228                        9387.105776
1            9944.511093                        9522.117209
2           10026.552404                        9638.838967
3           10102.500536                        9725.061514
4           10171.089470                        9795.591553
5           10234.678580                        9859.208726
6           10295.777129                        9919.353657
7           10355.659472                        9977.627970
8           10414.910193                       10034.912334
9           10473.827007                       10091.682494
10          10532.569067                       10148.185659
11          10591.220215                       10204.549635


Ahora estas sí son las predicciones en formato dólares

In [73]:
from statsmodels.stats.stattools import durbin_watson

# Calcular los estadísticos de Durbin-Watson para cada ecuación en el VECM
dw_stats_final_pca = [durbin_watson(vecm_final_pca_fit.resid[:, i]) for i in range(vecm_final_pca_fit.neqs)]

# Mostrar los estadísticos de Durbin-Watson
print("DURBIN WAT PRECIO INT __ D.W PRECIO DE EXP")
dw_stats_final_pca


DURBIN WAT PRECIO INT __ D.W PRECIO DE EXP


[2.0152651822735397, 1.9878186767203974]

In [74]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt



X = df[['CAC_INTERNACIONAL_TN', 'CAC_PRECIO_DE_EXPORTACION_ECUADOR']]
y = df['CAC_EXPORTACIONES_FOB']

# Divide los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarización de las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instancia y entrena el modelo LASSO
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)

# Predicciones
y_train_pred = lasso.predict(X_train_scaled)
y_test_pred = lasso.predict(X_test_scaled)

# Cálculo del R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R2:", r2_train+resid)

# Obtener los coeficientes
coeficientes = lasso.coef_
intercept = lasso.intercept_


# Función para predecir nuevas entradas
def predict_cac_exportaciones(cac_precio_tn, cac_precio_venta_ecuador_tn):
    new_data = pd.DataFrame({
        'CAC_INTERNACIONAL_TN': [cac_precio_tn],
        'CAC_PRECIO_DE_EXPORTACION_ECUADOR': [cac_precio_venta_ecuador_tn]
    })

    # Estandarizar las nuevas entradas
    new_data_scaled = scaler.transform(new_data)

    # Predecir usando el modelo LASSO
    predicted_exportaciones = lasso.predict(new_data_scaled)[0]
    return predicted_exportaciones


data = {
    "Coef": ["Intercepto", "Coeficiente de CAC_INTERNACIONAL_TN", "Coeficiente de CAC_PRECIO_DE_EXPORTACION_ECUADOR", "Media de CAC_INTERNACIONAL_TN", "Desviación estándar de CAC_INTERNACIONAL_TN", "Media de CAC_PRECIO_DE_EXPORTACION_ECUADOR", "Desviación estándar de CAC_PRECIO_DE_EXPORTACION_ECUADOR"],
    "Valor": [67754139.68, 20172354.58, 9341215.26, 2621.27, 720.42, 2775.43, 205.98]
}

df_resumen = pd.DataFrame(data)

df_resumen


R2: 0.7366751936296616


,Coef,Valor
0,Intercepto,67754139.68
1,Coeficiente de CAC_INTERNACIONAL_TN,20172354.58
2,Coeficiente de CAC_PRECIO_DE_EXPORTACION_ECUADOR,9341215.26
3,Media de CAC_INTERNACIONAL_TN,2621.27
4,Desviación estándar de CAC_INTERNACIONAL_TN,720.42
5,Media de CAC_PRECIO_DE_EXPORTACION_ECUADOR,2775.43
6,Desviación estándar de CAC_PRECIO_DE_EXPORTACI...,205.98


In [75]:
## REEMPLAZAR LOS PRECIOS

cac_internacional = 8123.471768  #REEMPLAZAR X1 CON PRECIO INTERNACIONAL
cac_nacional = 7188.326653  #REEMPLAZAR X2 CON PRECIO DE EXPORTACION ECUADOR

tn = (67754139.68+(20172354.58*((cac_internacional-2621.27)/742.42)+(934124.26*((cac_nacional-2775.43)/205.98))))
toneladas_proyectadas = tn/cac_nacional


print("Toneladas proyectadas a esos precios es")
print(toneladas_proyectadas)


Toneladas proyectadas a esos precios es
33007.335122425546


In [76]:
# Función para calcular la elasticidad precio-cantidad
def calcular_elasticidad_precio_cantidad(x, y):
    # Aplicar logaritmo natural a las variables
    log_x = np.log(df[x].dropna())
    log_y = np.log(df[y].dropna())

    # Asegurarse de tener el mismo número de observaciones
    min_len = min(len(log_x), len(log_y))
    log_x = log_x[:min_len]
    log_y = log_y[:min_len]

    # Ajustar un modelo lineal
    beta, alpha = np.polyfit(log_x, log_y, 1)
    return beta

# Calcular la elasticidad precio-cantidad
elasticidad_precio_cantidad = calcular_elasticidad_precio_cantidad('CAC_INTERNACIONAL_TN', 'CAC_EXPORTACIONES_TN')

# Crear tabla de resultados
resultados_elasticidad = {
    "Técnica": ["Elasticidad del precio int vs exportaciones", "Elasticidad del precio Ecuador vs Exportaciones"],
    "Resultado": [elasticidad_precio_precio, elasticidad_precio_cantidad]
}

resultados_elasticidad_df = pd.DataFrame(resultados_elasticidad)

# Mostrar resultados
resultados_elasticidad_df

,Técnica,Resultado
0,Elasticidad del precio int vs exportaciones,0.015726
1,Elasticidad del precio Ecuador vs Exportaciones,0.028022


Pearson, correlacion entre Precio y Toneladas

In [84]:
CORRELACION = df['CAC_INTERNACIONAL_TN'].corr(df['CAC_EXPORTACIONES_TN'])


CORRELACION

0.052714960920690966

In [86]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Paso 1: Regresión de las dos series
X = df['CAC_INTERNACIONAL_TN']
y = df['CAC_EXPORTACIONES_TN']

# Agregamos una constante a X
X = sm.add_constant(X)

# Realizamos la regresión
model = sm.OLS(y, X).fit()
residuals = model.resid

# Paso 2: Aplicamos el ADF test a los residuos
adf_test = adfuller(residuals)

# Imprimimos los resultados
print("ADF Test Statistic:", adf_test[0])
print("p-value:", adf_test[1])
print("Critical Values:")
for key, value in adf_test[4].items():
    print(f"   {key}: {value}")



ADF Test Statistic: -0.7956007312196388
p-value: 0.8204109051154487
Critical Values:
   1%: -3.4912451337340342
   5%: -2.8881954545454547
   10%: -2.5809876033057852


no hay evidencia de cointegración entre CAC_INTERNACIONAL_TN y CAC_EXPORTACIONES_TN según la prueba de Engle-Granger. Esto sugiere que no existe una relación de equilibrio a largo plazo entre estas dos variables.

In [89]:
import statsmodels.api as sm

# Definir las variables
X = df['CAC_INTERNACIONAL_TN']  # Variable independiente
y = df['CAC_EXPORTACIONES_TN']   # Variable dependiente

# Agregar una constante a X para el intercepto
X = sm.add_constant(X)

# Realizar la regresión lineal
model = sm.OLS(y, X).fit()

# Imprimir el resumen de la regresión
print(model.summary())


                             OLS Regression Results                             
Dep. Variable:     CAC_EXPORTACIONES_TN   R-squared:                       0.003
Model:                              OLS   Adj. R-squared:                 -0.006
Method:                   Least Squares   F-statistic:                    0.3344
Date:                  Mon, 12 Aug 2024   Prob (F-statistic):              0.564
Time:                          21:33:54   Log-Likelihood:                -1293.8
No. Observations:                   122   AIC:                             2592.
Df Residuals:                       120   BIC:                             2597.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const       

El coeficiente estimado para CAC_INTERNACIONAL_TN es de 0.4292, lo que sugiere una relación muy débil entre las dos variables. Además, el p-valor asociado a este coeficiente es 0.564, que es significativamente mayor que el umbral comúnmente aceptado de 0.05. Esto significa que no podemos rechazar la hipótesis nula de que el coeficiente es cero, indicando que cualquier efecto observado podría ser resultado del azar. Asimismo, el intervalo de confianza del 95% para el coeficiente de CAC_INTERNACIONAL_TN es [-1.040, 1.899], lo cual incluye el valor cero, reforzando la conclusión de que el precio internacional del cacao no afecta de manera significativa las exportaciones de cacao desde Ecuador.